In [1]:
import pyspark
import numpy as np
import pandas as pd

data = pd.read_csv("Structured_data2", sep=';')

In [2]:
data.head(5)

,Scrap_date,Scrap_time,Country_from,Country_to,Flight_id,Days,Journey_time,Airline1_There,Airline1_Back,Airline2_There,...,Depart_from2_Back,Arrival_hour1_There,Arrival_hour1_Back,Arrival_hour2_There,Arrival_hour2_Back,Arrive_to1_There,Arrive_to1_Back,Arrive_to2_There,Arrive_to2_Back,Full_Price
0,2019-05-15,2,Austria,Austria,71,147,8,Laudamotion,Laudamotion,Laudamotion,...,Palma Mall PMI,afternoon,morning,evening,afternoon,Palma Mall PMI OE542,Palma Mall PMI OE502,Vienna VIE,Salzburg SZG,99.96
1,2019-05-15,2,Austria,Belgium,265,2,7,Laudamotion,Jetairfly,Jetairfly,...,Heraklion HER,afternoon,afternoon,evening,afternoon,Palma Mall PMI OE122,Heraklion HER TB2281,Ostend OSTOstend-Bruges OST,Vienna VIE,183.80
2,2019-05-15,2,Austria,Croatia,345,41,8,Laudamotion,easyJet,Volotea,...,Berlin SXFBerlin (Schonefeld) SXF,morning,morning,evening,evening,Milan BGYMilan (Bergamo) BGY OE1318,Berlin SXFBerlin (Schonefeld) SXF U24756,Split SPU,Vienna VIE,63.04
3,2019-05-15,2,Austria,Denmark,526,22,6,Laudamotion,Wizz Air,Ryanair,...,none,morning,afternoon,afternoon,none,Milan BGYMilan (Bergamo) BGY OE1318,Vienna VIE,Billund BLL,none,47.40
4,2019-05-15,2,Austria,England,659,21,8,Laudamotion,Ryanair,none,...,Milan MXPMilan (Malpensa) MXP,morning,morning,none,evening,London STNLondon (Stansted) STN,Milan MXPMilan (Malpensa) MXP FR8729,none,Vienna VIE,45.34


In [3]:
np.mean(data.loc[data['Airline1_There']=='Czech Airlines']['Price1_There'])

108.54778489830036

In [4]:
data['Airline1_Back'].drop_duplicates()

0              Laudamotion
1                Jetairfly
2                  easyJet
3                 Wizz Air
4                  Ryanair
7                Eurowings
12                 Vueling
21               norwegian
30                 Volotea
36              SmartWings
47          Czech Airlines
57               airBaltic
71                 Pegasus
90                   Flybe
100             Aer Lingus
113                 Pobeda
299                Widerøe
467                   Jet2
495              Air Malta
577       Transavia France
1055                Aegean
1661           Thomas Cook
1665                Condor
2379             Transavia
4962          Level Anisec
6721         Blue Panorama
9651              Blue Air
10450               TUIfly
14652           SunExpress
50640           Thomsonfly
430625           Red Wings
Name: Airline1_Back, dtype: object

## Tu się zaczyna Spark

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ml-flights').getOrCreate()
df = spark.read.csv("Structured_data2", sep=';', header = True, inferSchema = True)
df.printSchema()

root
 |-- Scrap_date: timestamp (nullable = true)
 |-- Scrap_time: integer (nullable = true)
 |-- Country_from: string (nullable = true)
 |-- Country_to: string (nullable = true)
 |-- Flight_id: integer (nullable = true)
 |-- Days: integer (nullable = true)
 |-- Journey_time: integer (nullable = true)
 |-- Airline1_There: string (nullable = true)
 |-- Airline1_Back: string (nullable = true)
 |-- Airline2_There: string (nullable = true)
 |-- Airline2_Back: string (nullable = true)
 |-- Price1_There: double (nullable = true)
 |-- Price1_Back: double (nullable = true)
 |-- Price2_There: double (nullable = true)
 |-- Price2_Back: double (nullable = true)
 |-- Depart_hour1_There: string (nullable = true)
 |-- Depart_hour1_Back: string (nullable = true)
 |-- Depart_hour2_There: string (nullable = true)
 |-- Depart_hour2_Back: string (nullable = true)
 |-- Depart_from1_There: string (nullable = true)
 |-- Depart_from1_Back: string (nullable = true)
 |-- Depart_from2_There: string (nullable = 

In [6]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
Scrap_date,2019-05-15 00:00:00,2019-05-15 00:00:00,2019-05-15 00:00:00,2019-05-15 00:00:00,2019-05-15 00:00:00
Scrap_time,2,2,2,2,2
Country_from,Austria,Austria,Austria,Austria,Austria
Country_to,Austria,Belgium,Croatia,Denmark,England
Flight_id,71,265,345,526,659
Days,147,2,41,22,21
Journey_time,8,7,8,6,8
Airline1_There,Laudamotion,Laudamotion,Laudamotion,Laudamotion,Laudamotion
Airline1_Back,Laudamotion,Jetairfly,easyJet,Wizz Air,Ryanair
Airline2_There,Laudamotion,Jetairfly,Volotea,Ryanair,none


In [7]:
from pyspark.sql import functions as func

means_airline1_t = df.groupBy("Airline1_There","Scrap_date").agg(func.mean("Price1_There"))
means_airline2_t = df.groupBy("Airline2_There","Scrap_date").agg(func.mean("Price2_There"))
means_airline1_b = df.groupBy("Airline1_Back","Scrap_date").agg(func.mean("Price1_Back"))
means_airline2_b = df.groupBy("Airline2_Back","Scrap_date").agg(func.mean("Price2_Back"))

# sprawdzone w pandasie, wychodzi to samo

In [8]:
means_airline1_t.withColumnRenamed("Airline1_There","Airline").withColumnRenamed("avg(Price1_There)","Price").sort(func.desc("Airline1_There")).show()

+---------+-------------------+------------------+
|  Airline|         Scrap_date|             Price|
+---------+-------------------+------------------+
|norwegian|2019-05-15 00:00:00| 52.84984296619399|
|norwegian|2019-05-16 00:00:00| 52.78001725625525|
|norwegian|2019-05-19 00:00:00|53.696963465324735|
|norwegian|2019-05-17 00:00:00| 53.05857079449609|
|norwegian|2019-05-18 00:00:00| 54.01742630385472|
|  easyJet|2019-05-17 00:00:00| 15.15546314741036|
|  easyJet|2019-05-19 00:00:00|15.117561431623923|
|  easyJet|2019-05-18 00:00:00|15.242173347057365|
|  easyJet|2019-05-15 00:00:00|15.180706949366286|
|  easyJet|2019-05-16 00:00:00|15.202175210518519|
|airBaltic|2019-05-19 00:00:00| 74.17188379646376|
|airBaltic|2019-05-17 00:00:00| 74.08182935647189|
|airBaltic|2019-05-16 00:00:00| 73.40133384734038|
|airBaltic|2019-05-15 00:00:00| 74.25292947558812|
|airBaltic|2019-05-18 00:00:00| 74.05884315117152|
| Wizz Air|2019-05-19 00:00:00|21.710859925788753|
| Wizz Air|2019-05-15 00:00:00|

In [9]:
m_1 = means_airline1_t.withColumnRenamed("Airline1_There","Airline").withColumnRenamed("avg(Price1_There)","Price")
m_2 = means_airline2_t.withColumnRenamed("Airline2_There","Airline").withColumnRenamed("avg(Price2_There)","Price")
m_3 = means_airline1_b.withColumnRenamed("Airline1_Back","Airline").withColumnRenamed("avg(Price1_Back)","Price")
m_4 = means_airline2_b.withColumnRenamed("Airline2_Back","Airline").withColumnRenamed("avg(Price2_Back)","Price")

means_airlines = m_1.union(m_2).union(m_3).union(m_4)

In [10]:
means_airlines.sort(func.desc("Airline")).show()

+---------+-------------------+------------------+
|  Airline|         Scrap_date|             Price|
+---------+-------------------+------------------+
|norwegian|2019-05-16 00:00:00|49.580249554367185|
|norwegian|2019-05-19 00:00:00|51.264948036951516|
|norwegian|2019-05-18 00:00:00| 53.86310307256472|
|norwegian|2019-05-18 00:00:00|50.401906392694066|
|norwegian|2019-05-19 00:00:00| 54.82445729537357|
|norwegian|2019-05-17 00:00:00| 48.87306786315198|
|norwegian|2019-05-15 00:00:00| 52.84984296619399|
|norwegian|2019-05-19 00:00:00|54.350380134886606|
|norwegian|2019-05-15 00:00:00| 47.35857603092783|
|norwegian|2019-05-15 00:00:00| 51.42171320921976|
|norwegian|2019-05-16 00:00:00| 52.21809258465484|
|norwegian|2019-05-17 00:00:00| 53.40838115863564|
|norwegian|2019-05-19 00:00:00|53.696963465324735|
|norwegian|2019-05-17 00:00:00| 52.88045221842993|
|norwegian|2019-05-16 00:00:00| 52.78001725625525|
|norwegian|2019-05-18 00:00:00| 54.01742630385472|
|norwegian|2019-05-18 00:00:00|

In [11]:
means_airlines = means_airlines.groupBy("Airline","Scrap_date").agg(func.mean("Price"))

In [12]:
means_airlines.sort(func.desc("Airline")).show()

+---------+-------------------+------------------+
|  Airline|         Scrap_date|        avg(Price)|
+---------+-------------------+------------------+
|norwegian|2019-05-17 00:00:00| 52.05511800867841|
|norwegian|2019-05-18 00:00:00|52.897461054954434|
|norwegian|2019-05-19 00:00:00| 53.53418723313411|
|norwegian|2019-05-16 00:00:00| 51.51969099616231|
|norwegian|2019-05-15 00:00:00| 50.26734074389308|
|     none|2019-05-15 00:00:00|               0.0|
|     none|2019-05-18 00:00:00|               0.0|
|     none|2019-05-16 00:00:00|               0.0|
|     none|2019-05-17 00:00:00|               0.0|
|     none|2019-05-19 00:00:00|               0.0|
|  easyJet|2019-05-16 00:00:00|16.833947433398706|
|  easyJet|2019-05-15 00:00:00| 16.79136297981047|
|  easyJet|2019-05-17 00:00:00|16.797516782214963|
|  easyJet|2019-05-19 00:00:00|16.892990652835568|
|  easyJet|2019-05-18 00:00:00|16.893608287074393|
|airBaltic|2019-05-16 00:00:00| 68.53146647377159|
|airBaltic|2019-05-17 00:00:00|

In [13]:
# poczytać więcej w jaki dokładnie sposób działają te przewidywania? 

In [14]:
df

DataFrame[Scrap_date: timestamp, Scrap_time: int, Country_from: string, Country_to: string, Flight_id: int, Days: int, Journey_time: int, Airline1_There: string, Airline1_Back: string, Airline2_There: string, Airline2_Back: string, Price1_There: double, Price1_Back: double, Price2_There: double, Price2_Back: double, Depart_hour1_There: string, Depart_hour1_Back: string, Depart_hour2_There: string, Depart_hour2_Back: string, Depart_from1_There: string, Depart_from1_Back: string, Depart_from2_There: string, Depart_from2_Back: string, Arrival_hour1_There: string, Arrival_hour1_Back: string, Arrival_hour2_There: string, Arrival_hour2_Back: string, Arrive_to1_There: string, Arrive_to1_Back: string, Arrive_to2_There: string, Arrive_to2_Back: string, Full_Price: double]

In [21]:
train_df = df.filter(func.col('Scrap_date') != '2019-05-15 00:00:00')
val_df = df.filter(func.col('Scrap_date') == '2019-05-15 00:00:00')

In [22]:
print((train_df.count(), len(train_df.columns)))
print((val_df.count(), len(val_df.columns)))

(547638, 32)
(136397, 32)


In [35]:
train_X = train_df.drop('Price1_There', 'Price2_There', 'Price1_Back', 'Price2_Back', 'Full_Price')
train_y = train_df.select('Full_Price')

In [36]:
train_X.printSchema()

root
 |-- Scrap_date: timestamp (nullable = true)
 |-- Scrap_time: integer (nullable = true)
 |-- Country_from: string (nullable = true)
 |-- Country_to: string (nullable = true)
 |-- Flight_id: integer (nullable = true)
 |-- Days: integer (nullable = true)
 |-- Journey_time: integer (nullable = true)
 |-- Airline1_There: string (nullable = true)
 |-- Airline1_Back: string (nullable = true)
 |-- Airline2_There: string (nullable = true)
 |-- Airline2_Back: string (nullable = true)
 |-- Depart_hour1_There: string (nullable = true)
 |-- Depart_hour1_Back: string (nullable = true)
 |-- Depart_hour2_There: string (nullable = true)
 |-- Depart_hour2_Back: string (nullable = true)
 |-- Depart_from1_There: string (nullable = true)
 |-- Depart_from1_Back: string (nullable = true)
 |-- Depart_from2_There: string (nullable = true)
 |-- Depart_from2_Back: string (nullable = true)
 |-- Arrival_hour1_There: string (nullable = true)
 |-- Arrival_hour1_Back: string (nullable = true)
 |-- Arrival_hour2

In [46]:
type(train_y)

pyspark.sql.dataframe.DataFrame

In [40]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType, ArrayType


# Make some fake data and train a model.
#n_samples_test = 100000
#n_samples_train = 1000
#n_samples_all = n_samples_train + n_samples_test
#n_features = 50

#X, y = make_classification(n_samples=n_samples_all, n_features=n_features, random_state=123)
#X_train, X_test, y_train, y_test = \
#    train_test_split(X, y, test_size=n_samples_test, random_state=45)

# Use pandas to put the test data in parquet format to illustrate how to load it up later.
# In real usage, the data might be on S3, Azure Blog Storage, HDFS, etc.
#column_names = [f'feature{i}' for i in range(n_features)]
#(
#    pd.DataFrame(X_test, columns=column_names)
#    .reset_index()
#    .rename(columns={'index': 'id'})
#    .to_parquet('unlabeled_data')
#)

#Train a model with scikit-learn

param_grid = {'n_estimators': [100], 'max_depth': [2, 4, None]}
gs_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='roc_auc'
).fit(train_X, train_y)
print('ROC AUC: %.3f' % gs_rf.best_score_)

#ROC AUC: 0.959

#Set up a spark environment
#In [4]:

#sc = pyspark.SparkContext(appName="foo")
#sqlContext = pyspark.SQLContext(sc)


# model jest trenowany dokładnie na wszystkich danych
# a potem predykcja jest robiona na odpowiednich kawałkach
# i potem to jest zgrywane do pliku CSV

# model powinien być zapisywany tak żeby się nie kasował po skończonej sesji
# spark ml umie zapisać na hdfs wyeksportowany model


/home/kasia/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ValueError: Expected array-like (array or non-string sequence), got DataFrame[Full_Price: double]

In [5]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']

In [13]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

categoricalColumns = ['Country_from', 'Country_to', 'Airline1_There', 'Airline1_Back', 
                      'Airline2_There', 'Airline2_Back', 'Depart_hour1_There', 'Depart_hour1_Back',
                      'Depart_hour2_There', 'Depart_hour2_Back','Depart_from1_There','Depart_from1_Back',
                      'Depart_from2_There','Depart_from2_Back','Arrival_hour1_There','Arrival_hour1_Back',
                      'Arrival_hour2_There','Arrival_hour2_Back','Arrive_to1_There','Arrive_to1_Back',
                      'Arrive_to2_There','Arrive_to2_Back']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

#label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
#stages += [label_stringIdx]

numericCols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

## Bonus - wykres pomiędzy którymi miejscami są najtańsze loty

Skonsultować czy na pewno dobrze rozumiem te nowe oznaczenia

In [13]:
means_places1_t = df.groupBy("Depart_from1_There","Arrive_to1_There").agg(func.mean("Full_Price"))
means_places2_t = df.groupBy("Depart_from2_There","Arrive_to2_There").agg(func.mean("Full_Price"))
means_places1_b = df.groupBy("Depart_from1_Back","Arrive_to1_Back").agg(func.mean("Full_Price"))
means_places2_b = df.groupBy("Depart_from2_Back","Arrive_to2_Back").agg(func.mean("Full_Price"))

In [14]:
means_places1_t.show()

+--------------------+--------------------+------------------+
|  Depart_from1_There|    Arrive_to1_There|   avg(Full_Price)|
+--------------------+--------------------+------------------+
|         Bologna BLQ|       Stuttgart STR| 19.98000000000001|
|          Dublin DUB|       Barcelona BCN| 55.42234374999998|
|          Bremen BRE|          Gdansk GDN|23.949776536312836|
|           Zadar ZAD|London LTNLondon ...| 87.39095485210173|
|            Lyon LYS|Berlin SXFBerlin ...| 48.34827160493828|
|          Dublin DUB|   Nantes NTE FR1986| 67.77999999999997|
|Moscow VKOMoscow ...|Berlin TXLBerlin ...| 97.72628125000003|
|       Barcelona BCN|Santander SDR FR6384|             40.97|
|Oslo OSLOslo (Gar...|          Gdansk GDN|19.163499999999996|
|          Athens ATH|Eindhoven EIN FR4307|           68.9975|
|      Copenhagen CPH|Ostersund OSD U22651| 55.58763157894738|
|Berlin SXFBerlin ...|            Pisa PSA|           28.6675|
|       Stavanger SVG|   Krakow KRK DY1036|            

In [15]:
means_places1_t = means_places1_t.sort(func.asc("avg(Full_Price)"))
means_places1_t.show()

+--------------------+--------------------+------------------+
|  Depart_from1_There|    Arrive_to1_There|   avg(Full_Price)|
+--------------------+--------------------+------------------+
|      Lamezia Te SUF|        Bordeaux BOD|10.622708333333335|
|Warsaw WMIWarsaw ...|Oslo TRFOslo (Tor...|11.782750000000004|
|Oslo TRFOslo (Tor...|Warsaw WMIWarsaw ...|11.826406250000005|
|        Biarritz BIQ|London STNLondon ...|11.995251798561151|
|Oslo TRFOslo (Tor...|          Gdansk GDN|12.327062500000002|
|          Gdansk GDN|Oslo TRFOslo (Tor...|12.770125000000002|
|           Porto OPO|          Malaga AGP|13.844999999999999|
|       Frankfurt FRA|            Pisa PSA|             13.98|
|        Bordeaux BOD|      Lamezia Te SUF| 14.48607476635514|
|        Toulouse TLS|London STNLondon ...|14.611062500000003|
|London STNLondon ...|        Toulouse TLS|15.010952380952384|
|Venice VCEVenice ...|           Ibiza IBZ|15.034285714285712|
|Venice VCEVenice ...|         Menorca MAH|            

In [17]:
means_places1_t.write.format('com.databricks.spark.csv').save('cheapest_connections.csv')